In [1]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sdss.metadata import MetaData
meta = MetaData()
%matplotlib inline

In [2]:
def specobjid_to_idx(specobjid: int, ids: np.array):
    mask = np.where(ids[:, 1]==specobjid, True, False)
    idx = int(ids[mask, 0][0])
    return idx

def set_intersection(specobjids: dict, max_rank: int, min_rank: int):

    """
    INPUT
    specobjids: dictionary with arrays of specobjids ordered by anomalous
        rank
        key: score name, e.g, lp_noRel100
        value: set with specobjid of spectra
    max_rank: the (max_rank+1)^{th} most anomalous spectrum
    min_rank: the (min_rank+1)^{th} most anomalous spectrum
        If the rank is 0 then it is the most anomalous spectrum
        If the rank is 1,then it is the second mosth anomalous spectrum...
    """
    score_names = list(specobjids.keys())
    # Adjust ranks
    if min_rank == 0:

        specobjids = {
            score_name: specobjids[score_name][-max_rank:] for score_name in score_names
            }

    else:

        specobjids = {
            score_name: specobjids[score_name][-max_rank:-min_rank] for score_name in score_names
            }
    
    # get a set of any score
    intersection_set = set(specobjids[score_names[0]])

    for score_name in score_names:

        intersection_set = intersection_set.intersection(
            set(specobjids[score_name])
        )
    
    return intersection_set

def set_difference(specobjids: set, intersection_specobjids: set):

    set_difference = specobjids.difference(intersection_specobjids)
    
    return set_difference

In [3]:
meta_data_directory = "/home/edgar/spectra/0_01_z_0_5_4_0_snr_inf"
scores_directory = f"{meta_data_directory}/bin_04/explanation/256_128_64/latent_12"
wave = np.load(f"{meta_data_directory}/wave.npy")
spectra = np.load(f"{meta_data_directory}/spectra.npy", mmap_mode="r")
ids = np.load(f"{meta_data_directory}/ids_inputting.npy")

## Get sets of specobjids of anomalous spectra per score

In [4]:
scores_names = {
    "correlation": "Correlation score",
    "correlation_filter_250kms": "Correlation score with a 250 kms filter",
    "cosine": "Cosine disimilarity score",
    "cosine_filter_250kms": "Cosine disimilarity score with a 250 kms filter",
    "lp_noRel100": "lp score",
    "lp_filter_250kms_noRel100": "lp score with a 250 kms filter",
    "lp_noRel97": "lp score ignoring 3% of largest residuals",
    "lp_filter_250kms_noRel97": "lp score ignoring 3% of largest residuals with a 250 kms filter",
    "lp_rel100": "lp relative score",
    "lp_filter_250kms_rel100": "lp relative score with a 250 kms filter",
    "lp_rel97": "lp relative score\n ignoring 3% of largest residuals",
    "lp_filter_250kms_rel97": "lp relative score ignoring 3% of largest residuals with a 250 kms filter",
    "mse_noRel100": "MSE score",
    "mse_filter_250kms_noRel100": "MSE score with a 250 kms filter",
    "mse_noRel97": "MSE score ignoring 3% of largest residuals",
    "mse_filter_250kms_noRel97": "MSE score ignoring 3% of largest residuals with a 250 kms filter",
    "mse_rel100": "MSE relative score",
    "mse_filter_250kms_rel100": "MSE relative score with a 250 kms filter",
    "mse_rel97": "MSE relative score\n ignoring 3% of largest residuals",
    "mse_filter_250kms_rel97": "MSE relative score ignoring 3% of largest residuals with a 250 kms filter",
    "mad_noRel100": "MAD score",
    "mad_filter_250kms_noRel100": "MAD score with a 250 kms filter",
    "mad_noRel97": "MAD score ignoring 3% of largest residuals",
    "mad_filter_250kms_noRel97": "MAD score ignoring 3% of largest residuals with a 250 kms filter",
    "mad_rel100": "MAD relative score",
    "mad_filter_250kms_rel100": "MAD relative score with a 250 kms filter",
    "mad_rel97": "MAD relative score\n ignoring 3% of largest residuals",
    "mad_filter_250kms_rel97": "MAD relative score ignoring 3% of largest residuals with a 250 kms filter"
}

In [5]:
scores_specobjid = {}

for score_name in scores_names.keys():
    
    specobjids = pd.read_csv(
        f"{scores_directory}/{score_name}/top_anomalies.csv.gz",
        index_col="specobjid",
    ).index
        
    scores_specobjid[score_name] = specobjids

### Intersection of all scores

In [6]:
max_rank = 10000
min_rank = 0
print(f"Max rank: {max_rank}, Min rank: {min_rank}")

number_of_anomalies = max_rank - min_rank
print(f"Number of anomalies: {number_of_anomalies}")

intersection_all_scores = set_intersection(
    scores_specobjid, max_rank=max_rank, min_rank=min_rank
)

percentage_intersection = (len(intersection_all_scores)/number_of_anomalies)*100

print(f"Percentage of common anomalies for all scores: {percentage_intersection:.2f}")

Max rank: 10000, Min rank: 0
Number of anomalies: 10000
Percentage of common anomalies for all scores: 0.16


## $L^p$ scores

In [7]:
lp_scores = [
    "lp_noRel100",
    "lp_noRel97",
    "lp_filter_250kms_noRel100",
    "lp_filter_250kms_noRel97",
    "lp_rel100",
    "lp_rel97",
    "lp_filter_250kms_rel100",
    "lp_filter_250kms_rel97"
    ]

lp_specobjids = {score_name: scores_specobjid[score_name] for score_name in lp_scores}

max_rank = 1000
min_rank = 0
print(f"Max rank: {max_rank}, Min rank: {min_rank}")

number_of_anomalies = max_rank - min_rank
print(f"Number of anomalies: {number_of_anomalies}")


intersection_all_scores = set_intersection(
    lp_specobjids, max_rank=max_rank, min_rank=min_rank
)
percentage_intersection = (len(intersection_all_scores)/number_of_anomalies)*100

print(f"Common anomalies for all lp scores: {percentage_intersection:.2f}%")


Max rank: 1000, Min rank: 0
Number of anomalies: 1000
Common anomalies for all lp scores: 2.50%


In [8]:
compare_lp = {
"lp_vs_lp_relative": ["lp_noRel100", "lp_rel100"],
"lp_vs_lp_filter" : ["lp_noRel100", "lp_filter_250kms_noRel100"],
"lp_vs_lp_filter_relative": ["lp_noRel100", "lp_filter_250kms_rel100"],
"lp_vs_lp_ignore": ["lp_noRel100", "lp_noRel97"],
"lp_vs_lp_relative_ignore": ["lp_noRel100", "lp_rel97"],
"lp_vs_lp_filter_ignore": ["lp_noRel100", "lp_filter_250kms_noRel97"],
"lp_vs_lp_filter_relative_ignore": ["lp_noRel100", "lp_filter_250kms_rel97"],
}
max_rank = 1000
min_rank = 0
print(f"Max rank: {max_rank}, Min rank: {min_rank}")

number_of_anomalies = max_rank - min_rank
print(f"Number of anomalies: {number_of_anomalies}")

for lp_comparison in compare_lp.keys():
    
    pair_of_sets = {
        score_name: scores_specobjid[score_name] for score_name in compare_lp[lp_comparison]
    }
    
    intersection_set = set_intersection(
        pair_of_sets, max_rank=max_rank, min_rank=min_rank)

    number_of_intesections = len(intersection_set)
    percentage_of_intersection = (number_of_intesections/number_of_anomalies)*100
    
    print(f"Intersection of {compare_lp[lp_comparison]}= {percentage_of_intersection:.2f}%")


Max rank: 1000, Min rank: 0
Number of anomalies: 1000
Intersection of ['lp_noRel100', 'lp_rel100']= 3.70%
Intersection of ['lp_noRel100', 'lp_filter_250kms_noRel100']= 88.60%
Intersection of ['lp_noRel100', 'lp_filter_250kms_rel100']= 3.70%
Intersection of ['lp_noRel100', 'lp_noRel97']= 92.00%
Intersection of ['lp_noRel100', 'lp_rel97']= 3.40%
Intersection of ['lp_noRel100', 'lp_filter_250kms_noRel97']= 86.40%
Intersection of ['lp_noRel100', 'lp_filter_250kms_rel97']= 3.40%


## MAD scores

In [9]:
mad_scores = [
    "mad_noRel100",
    "mad_noRel97",
    "mad_filter_250kms_noRel100",
    "mad_filter_250kms_noRel97",
    "mad_rel100",
    "mad_rel97",
    "mad_filter_250kms_rel100",
    "mad_filter_250kms_rel97"
    ]
mad_specobjids = {score_name: scores_specobjid[score_name] for score_name in mad_scores}

max_rank = 1000
min_rank = 0
print(f"Max rank: {max_rank}, Min rank: {min_rank}")

number_of_anomalies = max_rank - min_rank
print(f"Number of anomalies: {number_of_anomalies}")

intersection_all_scores = set_intersection(
    mad_specobjids, max_rank=max_rank, min_rank=min_rank
)
percentage_intersection = (len(intersection_all_scores)/number_of_anomalies)*100

print(f"Common anomalies for all MAD scores: {percentage_intersection:.2f}%")


Max rank: 1000, Min rank: 0
Number of anomalies: 1000
Common anomalies for all MAD scores: 27.00%


In [10]:
compare_mad = {
"mad_vs_mad_relative": ["mad_noRel100", "mad_rel100"],
"mad_vs_mad_filter" : ["mad_noRel100", "mad_filter_250kms_noRel100"],
"mad_vs_mad_filter_relative": ["mad_noRel100", "mad_filter_250kms_rel100"],
"mad_vs_mad_ignore": ["mad_noRel100", "mad_noRel97"],
"mad_vs_mad_relative_ignore": ["mad_noRel100", "mad_rel97"],
"mad_vs_mad_filter_ignore": ["mad_noRel100", "mad_filter_250kms_noRel97"],
"mad_vs_mad_filter_relative_ignore": ["mad_noRel100", "mad_filter_250kms_rel97"],
}

max_rank = 1000
min_rank = 0
print(f"Max rank: {max_rank}, Min rank: {min_rank}")

number_of_anomalies = max_rank - min_rank
print(f"Number of anomalies: {number_of_anomalies}")

for mad_comparison in compare_mad.keys():
    
    pair_of_sets = {
        score_name: scores_specobjid[score_name] for score_name in compare_mad[mad_comparison]
    }
    
    intersection_set = set_intersection(
        pair_of_sets, max_rank=max_rank, min_rank=min_rank
    )

    number_of_intesections = len(intersection_set)
    percentage_of_intersection = (number_of_intesections/(number_of_anomalies))*100
    
    print(f"Intersection of {compare_mad[mad_comparison]}= {percentage_of_intersection:.2f}%")


Max rank: 1000, Min rank: 0
Number of anomalies: 1000
Intersection of ['mad_noRel100', 'mad_rel100']= 34.00%
Intersection of ['mad_noRel100', 'mad_filter_250kms_noRel100']= 78.60%
Intersection of ['mad_noRel100', 'mad_filter_250kms_rel100']= 32.30%
Intersection of ['mad_noRel100', 'mad_noRel97']= 75.70%
Intersection of ['mad_noRel100', 'mad_rel97']= 46.50%
Intersection of ['mad_noRel100', 'mad_filter_250kms_noRel97']= 68.70%
Intersection of ['mad_noRel100', 'mad_filter_250kms_rel97']= 45.10%


## MSE scores

In [11]:
mse_scores = [
    "mse_noRel100",
    "mse_noRel97",
    "mse_filter_250kms_noRel100",
    "mse_filter_250kms_noRel97",
    "mse_rel100",
    "mse_rel97",
    "mse_filter_250kms_rel100",
    "mse_filter_250kms_rel97"
    ]

mse_specobjids = {score_name: scores_specobjid[score_name] for score_name in mse_scores}

max_rank = 1000
min_rank = 0
print(f"Max rank: {max_rank}, Min rank: {min_rank}")

number_of_anomalies = max_rank - min_rank
print(f"Number of anomalies: {number_of_anomalies}")

intersection_all_scores = set_intersection(
    mse_specobjids, max_rank=max_rank, min_rank=min_rank
)
percentage_intersection = (len(intersection_all_scores)/number_of_anomalies)*100

print(f"Common anomalies for all MSE scores: {percentage_intersection:.2f}%")


Max rank: 1000, Min rank: 0
Number of anomalies: 1000
Common anomalies for all MSE scores: 19.50%


In [12]:
compare_mse = {
"mse_vs_mse_relative": ["mse_noRel100", "mse_rel100"],
"mse_vs_mse_filter" : ["mse_noRel100", "mse_filter_250kms_noRel100"],
"mse_vs_mse_filter_relative": ["mse_noRel100", "mse_filter_250kms_rel100"],
"mse_vs_mse_ignore": ["mse_noRel100", "mse_noRel97"],
"mse_vs_mse_relative_ignore": ["mse_noRel100", "mse_rel97"],
"mse_vs_mse_filter_ignore": ["mse_noRel100", "mse_filter_250kms_noRel97"],
"mse_vs_mse_filter_relative_ignore": ["mse_noRel100", "mse_filter_250kms_rel97"],
}

max_rank = 10000
min_rank = 0
print(f"Max rank: {max_rank}, Min rank: {min_rank}")

number_of_anomalies = max_rank - min_rank
print(f"Number of anomalies: {number_of_anomalies}")

for mse_comparison in compare_mse.keys():
    
    pair_of_sets = {
        score_name: scores_specobjid[score_name] for score_name in compare_mse[mse_comparison]
    }
    
    intersection_set = set_intersection(
        pair_of_sets, max_rank=max_rank, min_rank=min_rank
    )

    number_of_intesections = len(intersection_set)
    percentage_of_intersection = (number_of_intesections/number_of_anomalies)*100
    
    print(f"Intersection of {compare_mse[mse_comparison]}= {percentage_of_intersection:.2f}%")


Max rank: 10000, Min rank: 0
Number of anomalies: 10000
Intersection of ['mse_noRel100', 'mse_rel100']= 80.32%
Intersection of ['mse_noRel100', 'mse_filter_250kms_noRel100']= 79.55%
Intersection of ['mse_noRel100', 'mse_filter_250kms_rel100']= 68.75%
Intersection of ['mse_noRel100', 'mse_noRel97']= 65.32%
Intersection of ['mse_noRel100', 'mse_rel97']= 63.47%
Intersection of ['mse_noRel100', 'mse_filter_250kms_noRel97']= 62.06%
Intersection of ['mse_noRel100', 'mse_filter_250kms_rel97']= 59.68%


## Residuals based scores

In [13]:
residual_scores = lp_scores + mad_scores + mse_scores
residual_specobjids = {score_name: scores_specobjid[score_name] for score_name in residual_scores}

max_rank = 10000
min_rank = 0
print(f"Max rank: {max_rank}, Min rank: {min_rank}")

number_of_anomalies = max_rank - min_rank
print(f"Number of anomalies: {number_of_anomalies}")

intersection_all_scores = set_intersection(
    residual_specobjids, max_rank=max_rank, min_rank=min_rank
)
percentage_intersection = (len(intersection_all_scores)/number_of_anomalies)*100

print(f"Common anomalies for all tata scores: {percentage_intersection:.2f}%")


Max rank: 10000, Min rank: 0
Number of anomalies: 10000
Common anomalies for all tata scores: 2.41%


In [21]:
tata_scores = [
    "lp_noRel100",
    # "mad_noRel100",
    "mse_noRel100",
    # "lp_rel100",
    # "mad_rel100",
    # "mse_noRel97",

]
tata_specobjids = {score_name: scores_specobjid[score_name] for score_name in tata_scores}

max_rank = 100
min_rank = 0
print(f"Max rank: {max_rank}, Min rank: {min_rank}")

number_of_anomalies = max_rank - min_rank
print(f"Number of anomalies: {number_of_anomalies}")

intersection_all_scores = set_intersection(
    tata_specobjids, max_rank=max_rank, min_rank=min_rank
)
percentage_intersection = (len(intersection_all_scores)/number_of_anomalies)*100

print(f"Common anomalies for all tata scores: {percentage_intersection:.2f}%")


Max rank: 100, Min rank: 0
Number of anomalies: 100
Common anomalies for all tata scores: 23.00%


## Cosine scores

In [161]:
cosine_scores = [
    "cosine",
    "cosine_filter_250kms"
    ]

cosine_specobjids = {score_name: scores_specobjid[score_name] for score_name in cosine_scores}

max_rank = 100
min_rank = 0
print(f"Max rank: {max_rank}, Min rank: {min_rank}")

number_of_anomalies = max_rank - min_rank
print(f"Number of anomalies: {number_of_anomalies}")

intersection_all_scores = set_intersection(
    cosine_specobjids, max_rank=max_rank, min_rank=min_rank
)
percentage_intersection = (len(intersection_all_scores)/number_of_anomalies)*100

print(f"Common anomalies for all MSE scores: {percentage_intersection:.2f}%")


Max rank: 100, Min rank: 0
Number of anomalies: 100
Common anomalies for all MSE scores: 66.00%


## Correlation scores

In [162]:
correlation_scores = [
    "correlation",
    "correlation_filter_250kms"
    ]

correlation_specobjids = {score_name: scores_specobjid[score_name] for score_name in correlation_scores}

max_rank = 100
min_rank = 0
print(f"Max rank: {max_rank}, Min rank: {min_rank}")

number_of_anomalies = max_rank - min_rank
print(f"Number of anomalies: {number_of_anomalies}")

intersection_all_scores = set_intersection(
    correlation_specobjids, max_rank=max_rank, min_rank=min_rank
)
percentage_intersection = (len(intersection_all_scores)/number_of_anomalies)*100

print(f"Common anomalies for all MSE scores: {percentage_intersection:.2f}%")


Max rank: 100, Min rank: 0
Number of anomalies: 100
Common anomalies for all MSE scores: 59.00%
